In [6]:
import os 
import sys
import glob
import xml.etree.ElementTree as ET
from cv2 import imread, imwrite, imshow, waitKey
import cv2


In [7]:
sys.path.insert(0, '../')

In [8]:
from utils.misc_utils import mkdir_p
from utils.infer_utils import convert_bbox_format, get_crops, Rectangle

In [9]:
data_dir  = ''
save_dir = '../../data/Imagenet/ILSVRC/Curated'
annotation_dir = ''

In [5]:
video_names = os.listdir(annotation_dir)
total_videos = len(video_names)

NameError: name 'annotation_dir' is not defined

In [ ]:
def get_label(video):
    return os.path.join(save_dir, video)

In [ ]:
for idx,video in enumerate(video_names):
    video_path = os.path.join(annotation_dir, video)
  
    xml_files = glob.glob(os.path.join(video_path, '*.xml'))
    print("[{}/{}] ".format(idx, total_videos) + video)
    for xml in xml_files:
        tree = ET.parse(xml)
        root = tree.getroot()
        folder = root.find('folder').text
        folder = folder.split('/',1)[1]
        filename = root.find('filename').text
  
        img_file = os.path.join(data_dir, folder, filename + '.JPEG')
        
        img = None 
        bboxs = []
        for object in root.iter('object'):
            bbox = object.find('bndbox')
            x_max = float(bbox.find('xmax').text)
            x_min = float(bbox.find('xmin').text)
            y_max = float(bbox.find('ymax').text)
            y_min = float(bbox.find('ymin').text)
            
            width = x_max - x_min + 1
            height = y_max - y_min + 1
            bboxs.append([x_min, y_min, width, height])
        for idx, obj in enumerate(root.iter('object')):
            id = int(obj.find('trackid').text)
            class_name = obj.find('name').text
            track_save_dir = get_label(video)
            
            mkdir_p(track_save_dir)
            
            savename = os.path.join(track_save_dir, '{}.{:02d}.crop.x.jpg'.format(filename, int(id)))
  
            if(os.path.isfile(savename)): continue
            if img is None:
                img = imread(img_file,1)
            if img is None:
                print('img not present')
    
            
            target_box = convert_bbox_format(Rectangle(*bboxs[idx]), 'center-based')
            
           
         
            crop, _ = get_crops(img, target_box, size_z=127, size_x=255, context_amount=0.5)
            
            imwrite(savename, crop, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
            
            

In [ ]:
img_test = imread('../../data/Imagenet/ILSVRC/Data/VID/train/ILSVRC2017_VID_train_0000/ILSVRC2017_train_00079000/000028.JPEG',1)